In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [16]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='3',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [17]:
[print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}") for run in runs]

run id: 33b9a6b4fd934de9b83daeb7d33d768f, rmse: 6.3058
run id: 8abc91a299bd44538f9d179f662f3908, rmse: 6.3058
run id: 6bdd58d101474021af5085435f99cf4d, rmse: 6.3058
run id: 8b3e2eaba10b4ba7a0eca05772f20a8e, rmse: 6.3058
run id: fbfe68bbd9ad4a09a10d64d676f290ec, rmse: 6.3086


[None, None, None, None, None]

In [18]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [20]:
# register new model
run_id = "33b9a6b4fd934de9b83daeb7d33d768f"
model_uri = f"runs:/{run_id}/model"

mlflow.register_model(
    model_uri=model_uri,
    name="nyc-taxi-regressor"
)

Successfully registered model 'nyc-taxi-regressor'.
2023/05/31 11:55:48 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 1
Created version '1' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1685523348851, current_stage='None', description=None, last_updated_timestamp=1685523348851, name='nyc-taxi-regressor', run_id='33b9a6b4fd934de9b83daeb7d33d768f', run_link=None, source='file:///C:/Users/Neko/OneDrive/Desktop/learning/Homework/mlops_zoomcamp/experiment-tracking/mlruns/3/33b9a6b4fd934de9b83daeb7d33d768f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [23]:
# change model stage
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 1, stage: None


In [25]:
model_version = 1
stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=stage,
    archive_existing_versions=False
)

<ModelVersion: aliases=[], creation_timestamp=1685523348851, current_stage='Staging', description=None, last_updated_timestamp=1685523822040, name='nyc-taxi-regressor', run_id='33b9a6b4fd934de9b83daeb7d33d768f', run_link=None, source='file:///C:/Users/Neko/OneDrive/Desktop/learning/Homework/mlops_zoomcamp/experiment-tracking/mlruns/3/33b9a6b4fd934de9b83daeb7d33d768f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [26]:
# annotate models
from datetime import datetime

date = datetime.today().date()

client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"Model version {model_version} was transitioned to {stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1685523348851, current_stage='Staging', description='Model version 1 was transitioned to Staging on 2023-05-31', last_updated_timestamp=1685523927088, name='nyc-taxi-regressor', run_id='33b9a6b4fd934de9b83daeb7d33d768f', run_link=None, source='file:///C:/Users/Neko/OneDrive/Desktop/learning/Homework/mlops_zoomcamp/experiment-tracking/mlruns/3/33b9a6b4fd934de9b83daeb7d33d768f/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>

In [41]:
# decide which to move to production
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    if filename.endswith('.csv'):
        df = pd.read_csv(filename)

        df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
        df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)
    elif filename.endswith('.parquet'):
        df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df

def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')

    return dv.transform(train_dicts)

def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models://{name}/{stage}")
    y_pred = model.predict(X_test)

    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [34]:
df = read_dataframe(r"../../data/green_tripdata_2021-02.parquet")

In [29]:
client.download_artifacts(
    run_id=run_id,
    path='preprocessor',
    dst_path='.'
)

C:\Users\Neko\AppData\Local\Temp\ipykernel_18152\4002364889.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.download_artifacts`` is deprecated since 2.0. This method will be removed in a future release. Use ``mlflow.artifacts.download_artifacts`` instead.
  client.download_artifacts(


'c:\\Users\\Neko\\OneDrive\\Desktop\\learning\\Homework\\mlops_zoomcamp\\experiment-tracking\\preprocessor'

In [30]:
import pickle

with open("preprocessor/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [35]:
X_test = preprocess(df, dv)

In [36]:
target = 'duration'
y_test = df[target].values

In [43]:
%time test_model(name=model_name, stage=stage,X_test=X_test, y_test=y_test)

MlflowException: Not a proper models:/ URI: models://nyc-taxi-regressor/Staging. Models URIs must be of the form 'models:/<model_name>/<suffix>' or 'models:/<model_name>@<alias>' where suffix is a model version, stage, or the string 'latest' and where alias is a registered model alias. Only one of suffix or alias can be defined at a time.